In [1]:
import numpy as np
import torch as t
from torch.autograd import Variable

#Feeding the given data(in problem) as some variables
x1_info = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] 
p_info = [28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5]
p_satw = 10.0**(8.07131-1730.63/(20.0+233.426))
p_satd = 10.0**(7.43155-1554.679/(20.0+240.337))

def loss(a): #defining the loss function
    total_loss= 0.0
    for i in range (11):
        x1 = x1_info[i]
        p= p_info[i]
        x2= 1-x1
        p_norm = x1*p_satw*t.exp(a[0]*(a[1]*(x2)/(a[0]*x1+a[1]*(x2)))**2) + (x2)*p_satd*t.exp(a[1]*(a[0]*x1/(a[0]*x1+a[1]*(x2)))**2)
        total_loss = total_loss + (p_norm-p)**2
    return total_loss #returning the defined loss value 

alpha = 1 #giving the error value as aplha
A= Variable(t.tensor ([1.0, 1.0]), requires_grad=True)


while alpha >= 0.05:
        loss(A).backward()
        alpha = t.linalg.norm(A.grad)
        X=.2
        while loss(A-X*A.grad)>loss(A):
            X=.5*X
        with t.no_grad():
            A-= X*A.grad
            A.grad.zero_()
print ('Estimated A12 and A21:',A)
print ('Objective Value:',loss(A))
from math import exp

for i in range (11) :
    x1 = x1_info[i]
    x2= 1-x1
    p_norm = x1*p_satw*exp(A[0]*(A[1]*(x2)/(A[0]*x1+A[1]*(x2)))**2) + (x2)*p_satd*exp(A[1]*(A[0]*x1/(A[0]*x1+A[1]*(x2)))**2)
    print ('calculated value', round(p_norm,1))
print('Given Values of p',p_info)
print('By camparing these two Calculated values and given values, the output: Calculated data is close to the given values of data')




Estimated A12 and A21: tensor([1.9584, 1.6893], requires_grad=True)
Objective Value: tensor(0.6702, grad_fn=<AddBackward0>)
calculated value 28.8
calculated value 34.6
calculated value 36.5
calculated value 36.9
calculated value 36.9
calculated value 36.8
calculated value 36.4
calculated value 35.4
calculated value 32.9
calculated value 27.7
calculated value 17.5
Given Values of p [28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]
By camparing these two Calculated values and given values, the output: Calculated data is close to the given values of data


In [6]:
import sklearn.gaussian_process as gp
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def objective (X, mean = 0, var = 0.1):
    noise = np.random.normal(mean, var)
    return ((4 - (2.1* (X[0]**2)) + ((X[0]**4) /3))* (X[0]**2)) + (X[0]*X[1]) + ((-4 + 4*x[1]**2))

def imp(X_, X, model, k = 0.01):
    Y_mu, Y_sigma = model.predict(X, return_std = True)
    Y__= model.predict(X_)
    
with np.errstate (divide='ignore'):
    y_best = np.max(Y_sigma)
    Z = (Y_mu - y_best - k)/(Y_sigma)
    ei= ((Y_mu - y_best - k)*norm. cdf(Z)) + (Y_sigma*norm.pdf(Z))
    ei[Y_sigma == 0.0]= 0.0
return X_[np.argmax(ei)]
   
x1= np.random.uniform(-3,3,5)
x2= np.random.uniform(-2,2,5)
X= p.asarray([[x1], [x2]]).transpose().squeeze(1)
Y= np. asarray ([objective(x) for x in X])
       
kernel = gp.kernels.RBF()
model = gp. GaussianProcessRegressor (kernel=kernel,
                                      alpha=1e-4,
                                      n_restarts_optimizer=0,
                                      normalize_y=True)

n_iters = 10
k = 0.01
for i in range (n_iters):
       model.fit(x,Y)
       # Select next point using expected improvement
       x1 = mp.random.uniform(-3,3,5)
       x2 = np.random.uniform(-2,2,5)
       X= np. asarray ([[x1],[x2]]). transpose ().squeeze (1)
       x_next = imp(X_= X_,X = X, model = model)
       y_next = objective (x_next)
       X.stack (x_next)
       Y.append (y_next)


NameError: name 'Y_sigma' is not defined